In [18]:
# =========================================================
# Imports and file paths
# =========================================================
import os
import scipy.io as sio
import h5py
import numpy as np
from sklearn.preprocessing import StandardScaler

project_folder = r"C:\Users\user\Desktop\si_project"

mat_files = [
    os.path.join(project_folder, 'dataset1.mat'),
    os.path.join(project_folder, 'dataset2.mat'),
    os.path.join(project_folder, 'dataset3.mat')
]

# =========================================================
#  Import datasets using helper function
# =========================================================
def import_data(mat_files):
    datasets = {}
    for f in mat_files:
        name = os.path.splitext(os.path.basename(f))[0]
        try:
            data = sio.loadmat(f)
            datasets[name] = data
            print(f"Loaded {f} using scipy.io")
        except NotImplementedError:
            try:
                with h5py.File(f, 'r') as hf:
                    datasets[name] = {k: hf[k][:] for k in hf.keys()}
                print(f"Loaded {f} using h5py")
            except Exception as e:
                print(f"Failed to read {f}: {e}")
        except Exception as e:
            print(f"Failed to read {f}: {e}")
    return datasets

datasets = import_data(mat_files)
print("\nAll loaded datasets:", list(datasets.keys()))

# =========================================================
#  Combine data into X (sensors) and Y (positions)
# =========================================================
def combine_data(datasets):
    X_list, y_list = [], []
    for name, data in datasets.items():
        sensor_key = [k for k in data.keys() if 'mag_sensors' in k][0]
        position_key = [k for k in data.keys() if 'tip_position' in k][0]
        X_list.append(np.array(data[sensor_key]))
        y_list.append(np.array(data[position_key]))
        print(f" {name}: {sensor_key} {X_list[-1].shape}, {position_key} {y_list[-1].shape}")
    X = np.vstack(X_list)
    y = np.vstack(y_list)
    print(f"\n Combined shapes -> X: {X.shape}, y: {y.shape}")
    return X, y

X, Y = combine_data(datasets)

# =========================================================
#  Optional: create DataFrame for convenience
# =========================================================
import pandas as pd
from itertools import combinations

sensor_cols = [f'sensor_{i+1}' for i in range(X.shape[1])]
coord_cols = ['x', 'y', 'z']

df = pd.DataFrame(
    np.hstack([X, Y]),
    columns=sensor_cols + coord_cols
)

print(df.head())

# =========================================================
#  Generate feature combinations (optional for subset search)
# =========================================================
n_features = len(sensor_cols)
all_combinations = []
for k in range(n_features - 2, n_features + 1):
    all_combinations.extend(combinations(sensor_cols, k))

print(f"Total subsets with almost all sensors: {len(all_combinations)}")
print("Example of first subsets:", all_combinations[:5])

Loaded C:\Users\user\Desktop\si_project\dataset1.mat using scipy.io
Loaded C:\Users\user\Desktop\si_project\dataset2.mat using scipy.io
Loaded C:\Users\user\Desktop\si_project\dataset3.mat using scipy.io

All loaded datasets: ['dataset1', 'dataset2', 'dataset3']
 dataset1: mag_sensors (2000, 12), tip_position (2000, 3)
 dataset2: mag_sensors2 (2000, 12), tip_position2 (2000, 3)
 dataset3: mag_sensors3 (2000, 12), tip_position3 (2000, 3)

 Combined shapes -> X: (6000, 12), y: (6000, 3)
   sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  sensor_7  \
0   15544.0   15471.0   13962.0   13618.0   13691.0   13941.0   15588.0   
1   14100.0   14144.0   14144.0   13692.0   13338.0   13517.0   13987.0   
2   14240.0   14266.0   14445.0   13859.0   13618.0   13756.0   14080.0   
3   14956.0   14949.0   14123.0   13707.0   13765.0   13954.0   14554.0   
4   14196.0   14246.0   16177.0   14830.0   13836.0   13911.0   13988.0   

   sensor_8  sensor_9  sensor_10  sensor_11  sensor_12     

In [19]:
# =========================================================
# Create DataFrame with sensors and coordinates
# =========================================================
import pandas as pd
from itertools import combinations
import numpy as np

# Sensor and coordinate column names
sensor_cols = [f'sensor_{i+1}' for i in range(X.shape[1])]
coord_cols = ['x', 'y', 'z']

# Create DataFrame
df = pd.DataFrame(
    np.hstack([X, Y]),  # stack sensors and positions
    columns=sensor_cols + coord_cols
)

print(df.head())

   sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  sensor_7  \
0   15544.0   15471.0   13962.0   13618.0   13691.0   13941.0   15588.0   
1   14100.0   14144.0   14144.0   13692.0   13338.0   13517.0   13987.0   
2   14240.0   14266.0   14445.0   13859.0   13618.0   13756.0   14080.0   
3   14956.0   14949.0   14123.0   13707.0   13765.0   13954.0   14554.0   
4   14196.0   14246.0   16177.0   14830.0   13836.0   13911.0   13988.0   

   sensor_8  sensor_9  sensor_10  sensor_11  sensor_12          x          y  \
0   15234.0   14156.0    13758.0    14098.0    14296.0  71.417546  -4.340464   
1   13951.0   14162.0    13774.0    13784.0    13931.0  24.514493  47.778850   
2   14036.0   14509.0    13958.0    13996.0    14122.0   2.951708  24.908397   
3   14404.0   14208.0    13798.0    14127.0    14282.0  46.508669  -0.243722   
4   13984.0   15788.0    14690.0    14185.0    14257.0 -41.687301  34.183930   

            z  
0   76.317760  
1  110.173076  
2  112.971805  
3  1

In [20]:
# Gerar combinações de quase todos os sensores 
n_features = len(sensor_cols)
all_combinations = []
for k in range(n_features - 2, n_features + 1):  # subsets of size 10, 11, 12
    all_combinations.extend(combinations(sensor_cols, k))

print(f"Total de subsets com quase todos os sensores: {len(all_combinations)}")
print("Exemplo de primeiros subsets:", all_combinations[:5])

Total de subsets com quase todos os sensores: 79
Exemplo de primeiros subsets: [('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10'), ('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_11'), ('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_12'), ('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_10', 'sensor_11'), ('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_10', 'sensor_12')]


In [21]:
# =========================================================
# Create DataFrame with sensors and coordinates
# =========================================================
import pandas as pd
from itertools import combinations
import numpy as np

# Sensor and coordinate column names
sensor_cols = [f'sensor_{i+1}' for i in range(X.shape[1])]
coord_cols = ['x', 'y', 'z']

# Create DataFrame
df = pd.DataFrame(
    np.hstack([X, Y]),  # stack sensors and positions
    columns=sensor_cols + coord_cols
)

print(df.head())

   sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  sensor_7  \
0   15544.0   15471.0   13962.0   13618.0   13691.0   13941.0   15588.0   
1   14100.0   14144.0   14144.0   13692.0   13338.0   13517.0   13987.0   
2   14240.0   14266.0   14445.0   13859.0   13618.0   13756.0   14080.0   
3   14956.0   14949.0   14123.0   13707.0   13765.0   13954.0   14554.0   
4   14196.0   14246.0   16177.0   14830.0   13836.0   13911.0   13988.0   

   sensor_8  sensor_9  sensor_10  sensor_11  sensor_12          x          y  \
0   15234.0   14156.0    13758.0    14098.0    14296.0  71.417546  -4.340464   
1   13951.0   14162.0    13774.0    13784.0    13931.0  24.514493  47.778850   
2   14036.0   14509.0    13958.0    13996.0    14122.0   2.951708  24.908397   
3   14404.0   14208.0    13798.0    14127.0    14282.0  46.508669  -0.243722   
4   13984.0   15788.0    14690.0    14185.0    14257.0 -41.687301  34.183930   

            z  
0   76.317760  
1  110.173076  
2  112.971805  
3  1

In [22]:
#preparação para treino de modelos
y = df[coord_cols]

# Exemplo: primeiro subset
subset = all_combinations[0]
X_subset = df[list(subset)]
print("Subset de features:", subset)
print(X_subset.head())

Subset de features: ('sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10')
   sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  sensor_7  \
0   15544.0   15471.0   13962.0   13618.0   13691.0   13941.0   15588.0   
1   14100.0   14144.0   14144.0   13692.0   13338.0   13517.0   13987.0   
2   14240.0   14266.0   14445.0   13859.0   13618.0   13756.0   14080.0   
3   14956.0   14949.0   14123.0   13707.0   13765.0   13954.0   14554.0   
4   14196.0   14246.0   16177.0   14830.0   13836.0   13911.0   13988.0   

   sensor_8  sensor_9  sensor_10  
0   15234.0   14156.0    13758.0  
1   13951.0   14162.0    13774.0  
2   14036.0   14509.0    13958.0  
3   14404.0   14208.0    13798.0  
4   13984.0   15788.0    14690.0  


In [23]:
# =========================================================
#  AUTOENCODER FEATURE EXTRACTION BEFORE anfis
# =========================================================
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data for AE training
X_train_ae, X_val_ae = train_test_split(X, test_size=0.2, random_state=42)

# Scale input data
scaler_ae = StandardScaler()
X_train_scaled = scaler_ae.fit_transform(X_train_ae)
X_val_scaled = scaler_ae.transform(X_val_ae)

X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32)

# ---------------------------------------------------------
# Define Autoencoder
# ---------------------------------------------------------
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=6):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

# ---------------------------------------------------------
# Train Autoencoder
# ---------------------------------------------------------
input_dim = X.shape[1]
latent_dim = 10  # you can tune this
ae = Autoencoder(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(ae.parameters(), lr=1e-3)
epochs = 50
batch_size = 32

for epoch in range(epochs):
    ae.train()
    permutation = torch.randperm(X_train_t.size(0))
    epoch_loss = 0
    for i in range(0, X_train_t.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x = X_train_t[indices]
        optimizer.zero_grad()
        outputs = ae(batch_x)
        loss = criterion(outputs, batch_x)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch+1) % 10 == 0:
        ae.eval()
        with torch.no_grad():
            val_loss = criterion(ae(X_val_t), X_val_t).item()
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss/len(X_train_t):.6f}, Val Loss: {val_loss:.6f}")

# ---------------------------------------------------------
# Extract latent features for entire dataset
# ---------------------------------------------------------
ae.eval()
with torch.no_grad():
    X_torch = torch.tensor(scaler_ae.transform(X), dtype=torch.float32)
    X_latent_t = ae.encoder(X_torch)
    X_latent = X_latent_t.numpy()

print(f" Latent features extracted. Original dim: {X.shape[1]} → Latent dim: {X_latent.shape[1]}")


Epoch [10/50], Train Loss: 0.000056, Val Loss: 0.001469
Epoch [20/50], Train Loss: 0.000035, Val Loss: 0.000919
Epoch [30/50], Train Loss: 0.000026, Val Loss: 0.000798
Epoch [40/50], Train Loss: 0.000019, Val Loss: 0.000498
Epoch [50/50], Train Loss: 0.000016, Val Loss: 0.000420
 Latent features extracted. Original dim: 12 → Latent dim: 10


In [25]:
# =========================================================
#  ANFIS MODEL USING AUTOENCODER LATENT FEATURES
#  
# =========================================================
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
import random, math, time

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device("cpu")  

# =========================================================
#  Normalize target coordinates 
# =========================================================
y_scaler = StandardScaler()
Y_scaled = y_scaler.fit_transform(Y)

# =========================================================
#  ANFIS architecture
# =========================================================
class ANFIS(nn.Module):
    def __init__(self, n_inputs, n_rules, centers, sigmas):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_rules = n_rules
        self.centers = nn.Parameter(centers.clone().float().to(device))
        self.sigmas = nn.Parameter(sigmas.clone().float().to(device))
        self.consequents = nn.Parameter(torch.randn(n_rules, n_inputs + 1) * 0.1)

    def gaussian_mf(self, x):
        diff = (x.unsqueeze(1) - self.centers.unsqueeze(0)) / (self.sigmas.unsqueeze(0) + 1e-9)
        g = torch.exp(-0.5 * (diff ** 2).sum(dim=-1))
        return g

    def forward(self, x):
        batch = x.shape[0]
        firing = self.gaussian_mf(x)
        norm = firing / (firing.sum(dim=1, keepdim=True) + 1e-9)
        x_aug = torch.cat([x, torch.ones(batch, 1, device=x.device)], dim=1)
        rule_out = torch.matmul(x_aug, self.consequents.t())
        y = (norm * rule_out).sum(dim=1, keepdim=True)
        return y, norm, rule_out

# =========================================================
#  Helper: initialize centers/sigmas using KMeans
# =========================================================
def compute_centers_sigmas(X_np, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(X_np)
    centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32)
    sigmas_list = []
    for i in range(n_clusters):
        cluster_points = X_np[kmeans.labels_ == i]
        if len(cluster_points) > 1:
            sigma = np.std(cluster_points, axis=0)
        else:
            sigma = np.std(X_np, axis=0) * 0.5
        sigma[sigma == 0] = 1e-2
        sigmas_list.append(sigma)
    sigmas = torch.tensor(np.stack(sigmas_list), dtype=torch.float32)
    return centers, sigmas

# =========================================================
#  ANFIS training loop
# =========================================================
def train_anfis(model, X_train_t, y_train_t, epochs=50, lr=1e-3, verbose=False):
    model.to(device)
    X_train_t, y_train_t = X_train_t.to(device), y_train_t.to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        opt.zero_grad()
        y_pred, _, _ = model(X_train_t)
        loss = mse_loss(y_pred, y_train_t)
        loss.backward()
        opt.step()
        if verbose and ((epoch + 1) % 10 == 0 or epoch == 0):
            print(f"Epoch {epoch+1}/{epochs} Train MSE: {loss.item():.6f}")
    return model

# =========================================================
#  Evaluation metrics
# =========================================================
def evaluate_metrics_torch(y_true_t, y_pred_t):
    y_true = y_true_t.detach().cpu().numpy().ravel()
    y_pred = y_pred_t.detach().cpu().numpy().ravel()
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    nrmse = rmse / (y_true.max() - y_true.min()) if (y_true.max() - y_true.min()) != 0 else float('nan')
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "NRMSE(%)": nrmse * 100}

# =========================================================
#  Stage 1 – Feature subset evaluation
# =========================================================
def eval_anfis_subset(X_train_t, y_train_t, X_val_t, y_val_t,
                      n_clusters=3, lr=1e-3, epochs=60):
    centers, sigmas = compute_centers_sigmas(X_train_t.numpy(), n_clusters)
    model = ANFIS(X_train_t.shape[1], n_clusters, centers, sigmas)
    train_anfis(model, X_train_t, y_train_t, epochs=epochs, lr=lr)
    model.eval()
    with torch.no_grad():
        y_pred_val, _, _ = model(X_val_t.to(device))
    mse = float(((y_val_t.to(device) - y_pred_val)**2).mean().cpu().numpy())
    return mse

def evaluate_feature_subsets_anfis(X, Y, subset_sizes=None, max_subsets_per_size=10):
    if subset_sizes is None:
        subset_sizes = [X.shape[1]-2, X.shape[1]-1]
    n_features = X.shape[1]
    results = []
    for size in subset_sizes:
        feature_combinations = list(itertools.combinations(range(n_features), size))
        if len(feature_combinations) > max_subsets_per_size:
            feature_combinations = random.sample(feature_combinations, max_subsets_per_size)
        for subset in feature_combinations:
            X_sub = X[:, subset]
            X_train, X_val, y_train, y_val = train_test_split(X_sub, Y, test_size=0.2, random_state=42)
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)
            X_train_t = torch.tensor(X_train, dtype=torch.float32)
            y_train_t = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
            X_val_t = torch.tensor(X_val, dtype=torch.float32)
            y_val_t = torch.tensor(y_val.reshape(-1, 1), dtype=torch.float32)
            n_clusters = min(5, max(2, len(subset)))
            mse = eval_anfis_subset(X_train_t, y_train_t, X_val_t, y_val_t, n_clusters=n_clusters)
            results.append({"features": subset, "metric": mse})
    results = sorted(results, key=lambda x: x["metric"])
    return results

# =========================================================
#  Stage 2 – Main loop (grid search + retrain)
# =========================================================
subset_results_all = []
for coord in range(3):
    print(f"\n Evaluating feature subsets for coordinate {coord+1} (ANFIS)...")
    res = evaluate_feature_subsets_anfis(X_latent, Y_scaled[:, coord], max_subsets_per_size=8)
    subset_results_all.append(res)
    print(f"Top 3 subsets for coordinate {coord+1}:")
    for r in res[:3]:
        print(f"Features: {r['features']}, MSE: {r['metric']:.6f}")

param_grid = {"n_clusters": [3, 4, 5], "lr": [1e-2, 1e-3], "epochs": [40, 80]}
final_results = []

for coord in range(3):
    print(f"\n Parameter Grid Search for coordinate {coord+1} (ANFIS)...")
    top_subsets = [r["features"] for r in subset_results_all[coord][:3]]
    best_val_mse = float("inf")
    best_model_info = None

    for subset in top_subsets:
        X_sub = X_latent[:, subset]
        y_coord = Y_scaled[:, coord]
        X_train_full, X_test, y_train_full, y_test = train_test_split(X_sub, y_coord, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        X_train_t = torch.tensor(X_train, dtype=torch.float32)
        y_train_t = torch.tensor(y_train.reshape(-1,1), dtype=torch.float32)
        X_val_t = torch.tensor(X_val, dtype=torch.float32)
        y_val_t = torch.tensor(y_val.reshape(-1,1), dtype=torch.float32)
        X_test_t = torch.tensor(X_test, dtype=torch.float32)
        y_test_t = torch.tensor(y_test.reshape(-1,1), dtype=torch.float32)

        for ncl, lr, epochs in itertools.product(param_grid["n_clusters"], param_grid["lr"], param_grid["epochs"]):
            centers, sigmas = compute_centers_sigmas(X_train, ncl)
            model = ANFIS(X_train.shape[1], ncl, centers, sigmas)
            train_anfis(model, X_train_t, y_train_t, epochs=epochs, lr=lr)
            model.eval()
            with torch.no_grad():
                y_pred_val, _, _ = model(X_val_t)
            val_metrics = evaluate_metrics_torch(y_val_t, y_pred_val)
            if val_metrics["MSE"] < best_val_mse:
                best_val_mse = val_metrics["MSE"]
                best_model_info = {
                    "coord": coord+1,
                    "best_subset": subset,
                    "n_clusters": ncl,
                    "lr": lr,
                    "epochs": epochs,
                    "val_metrics": val_metrics,
                    "scaler": scaler,
                }

    # Retrain best model on train+val
    print("\nRetraining best model on combined train+val...")
    subset = best_model_info["best_subset"]
    X_sub_all = X_latent[:, subset]
    y_coord_all = Y_scaled[:, coord]
    X_trainval, X_test, y_trainval, y_test = train_test_split(X_sub_all, y_coord_all, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_trainval_s = scaler.fit_transform(X_trainval)
    X_test_s = scaler.transform(X_test)

    X_trainval_t = torch.tensor(X_trainval_s, dtype=torch.float32)
    y_trainval_t = torch.tensor(y_trainval.reshape(-1,1), dtype=torch.float32)
    X_test_t = torch.tensor(X_test_s, dtype=torch.float32)
    y_test_t = torch.tensor(y_test.reshape(-1,1), dtype=torch.float32)

    centers, sigmas = compute_centers_sigmas(X_trainval_s, best_model_info["n_clusters"])
    best_model = ANFIS(X_trainval_t.shape[1], best_model_info["n_clusters"], centers, sigmas)
    train_anfis(best_model, X_trainval_t, y_trainval_t, epochs=best_model_info["epochs"], lr=best_model_info["lr"], verbose=True)

    best_model.eval()
    with torch.no_grad():
        y_pred_test, _, _ = best_model(X_test_t)
    test_metrics = evaluate_metrics_torch(y_test_t, y_pred_test)

    best_model_info["test_metrics"] = test_metrics
    best_model_info["model"] = best_model
    final_results.append(best_model_info)

    print(f"\n Best ANFIS model for coordinate {coord+1}:")
    print({
        "coord": best_model_info["coord"],
        "best_subset": best_model_info["best_subset"],
        "n_clusters": best_model_info["n_clusters"],
        "lr": best_model_info["lr"],
        "epochs": best_model_info["epochs"],
        "val_metrics": best_model_info["val_metrics"],
        "test_metrics": best_model_info["test_metrics"]
    })

print("\n\nANFIS training + evaluation complete.")



 Evaluating feature subsets for coordinate 1 (ANFIS)...
Top 3 subsets for coordinate 1:
Features: (0, 1, 2, 3, 5, 6, 7, 8, 9), MSE: 0.352413
Features: (0, 2, 3, 4, 5, 6, 7, 8, 9), MSE: 0.432333
Features: (0, 1, 3, 4, 5, 6, 7, 8, 9), MSE: 0.478384

 Evaluating feature subsets for coordinate 2 (ANFIS)...
Top 3 subsets for coordinate 2:
Features: (0, 1, 3, 4, 5, 6, 7, 8, 9), MSE: 0.412333
Features: (0, 1, 2, 3, 4, 5, 6, 8, 9), MSE: 0.480278
Features: (0, 1, 3, 5, 6, 7, 8, 9), MSE: 0.499492

 Evaluating feature subsets for coordinate 3 (ANFIS)...
Top 3 subsets for coordinate 3:
Features: (0, 1, 2, 3, 4, 5, 6, 7, 9), MSE: 0.431533
Features: (0, 1, 2, 3, 5, 6, 7, 8, 9), MSE: 0.541953
Features: (0, 1, 2, 3, 4, 7, 8, 9), MSE: 0.564060

 Parameter Grid Search for coordinate 1 (ANFIS)...

Retraining best model on combined train+val...
Epoch 1/80 Train MSE: 1.220117
Epoch 10/80 Train MSE: 0.430053
Epoch 20/80 Train MSE: 0.188494
Epoch 30/80 Train MSE: 0.070825
Epoch 40/80 Train MSE: 0.036609
Epo